In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, f1_score, precision_score, recall_score,
                             roc_auc_score, precision_recall_curve, auc, confusion_matrix)
import json

# Load your data
df = pd.read_csv('/kaggle/input/concatenated-pca/concatenated_data_synthethic_PCA.csv', header=None, delimiter=',')

# Drop columns that are not features
X = df.iloc[:,:]  # Features
y = df.iloc[:, -4:]   # Target variable

# Combine the multilabel targets into a single multiclass target
unique_rows = np.unique(y, axis=0).tolist()
y['combined'] = y.apply(lambda row: unique_rows.index(list(row)), axis=1)
y = y['combined'].astype(int)

# Setup KFold for cross-validation
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Lists to store results from each fold
accuracies = []
f1_scores = []
precisions = []
recalls = []
roc_aucs = []
prc_aucs = []
tprs = []
fprs = []

all_fpr = np.linspace(0, 1, 100)
mean_tpr = 0.0
mean_roc_auc = 0.0

all_recall = np.linspace(0, 1, 100)
mean_precision = 0.0
mean_prc_auc = 0.0

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Perform k-fold cross-validation
for train_index, test_index in kf.split(X_scaled, y):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Initialize and train the logistic regression model
    model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
    model.fit(X_train, y_train)

    # Make predictions
    predictions = model.predict(X_test)
    proba = model.predict_proba(X_test)

    # Calculate metrics
    accuracies.append(accuracy_score(y_test, predictions))
    f1_scores.append(f1_score(y_test, predictions, average='macro'))
    precisions.append(precision_score(y_test, predictions, average='macro'))
    recalls.append(recall_score(y_test, predictions, average='macro'))
    roc_aucs.append(roc_auc_score(y_test, proba, multi_class='ovr', average='macro'))

    # Calculate TPR and FPR
    cm = confusion_matrix(y_test, predictions)
    fp = cm.sum(axis=0) - np.diag(cm)
    fn = cm.sum(axis=1) - np.diag(cm)
    tp = np.diag(cm)
    tn = cm.sum() - (fp + fn + tp)
    tpr = tp / (tp + fn)
    fpr = fp / (fp + tn)
    tprs.append(np.mean(tpr))
    fprs.append(np.mean(fpr))
    
    # Compute ROC curve for each class
    for i in range(len(unique_rows)):
        fpr, tpr, _ = roc_curve(y_test == i, proba[:, i])
        mean_tpr += np.interp(all_fpr, fpr, tpr)
        mean_tpr[0] = 0.0
        mean_roc_auc += auc(fpr, tpr)
        
    # Compute precision-recall curve for each class
    for i in range(len(unique_rows)):
        precision, recall, _ = precision_recall_curve(y_test == i, proba[:, i])
        mean_precision += np.interp(all_recall, recall[::-1], precision[::-1])
        mean_precision[0] = 1.0
        mean_prc_auc += auc(recall, precision)
        
# Calculate average of all metrics
average_accuracy = np.mean(accuracies)
average_f1 = np.mean(f1_scores)
average_precision = np.mean(precisions)
average_recall = np.mean(recalls)
average_roc_auc = np.mean(roc_aucs)
average_tpr = np.mean(tprs)
average_fpr = np.mean(fprs)

# Print summary of metrics
print(f"Average Accuracy: {average_accuracy:.10f}")
print(f"Average F1 Score: {average_f1:.10f}")
print(f"Average Precision: {average_precision:.10f}")
print(f"Average Recall: {average_recall:.10f}")
print(f"Average ROC AUC: {average_roc_auc:.10f}")
print(f"Average TPR: {average_tpr:.10f}")
print(f"Average FPR: {average_fpr:.10f}")


# Normalize mean_tpr and mean_roc_auc
mean_tpr /= (n_splits * len(unique_rows))
mean_roc_auc /= (n_splits * len(unique_rows))

# Plot micro-average ROC curve
plt.figure()
plt.plot(all_fpr, mean_tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % mean_roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) curve (micro-average)')
plt.legend(loc="lower right")
plt.savefig('logistic_regression_roc_curve_synthetic_PCA.jpg')
plt.show()

# Normalize mean_precision and mean_prc_auc
mean_precision /= (n_splits * len(unique_rows))
mean_prc_auc /= (n_splits * len(unique_rows))

# Plot micro-average precision-recall curve
plt.figure()
plt.plot(all_recall, mean_precision, color='darkorange', lw=2, label='PR curve (area = %0.2f)' % mean_prc_auc)
plt.plot([0, 1], [1, 0], color='black', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall curve (micro-average)')
plt.legend(loc="lower right")
plt.savefig('logistic_regression_prc_curve_synthetic_PCA.jpg')
plt.show()

# Save metrics to a JSON file
metrics = {
    'Average Accuracy': average_accuracy,
    'Average F1 Score': average_f1,
    'Average Precision': average_precision,
    'Average Recall': average_recall,
    'Average ROC AUC': average_roc_auc,
    'Average TPR': average_tpr,
    'Average FPR': average_fpr,
    'TPR List': tprs,
    'FPR List' : fprs,
    'Precision List' : precisions,
    "Recall List": recalls
}
with open('logistic_regression_evaluation_metrics_synthetic_PCA.json', 'w') as file:
    json.dump(metrics, file)

